In [15]:
%pip install beautifulsoup4 scikit-learn pandas

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 648.3 kB/s eta 0:00:00
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/106.4 kB ? eta -:--:--
   ----------------------- ---------------- 61.4/106.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 106.4/106.4 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
    --------------------------------------- 0.1/8.9 MB 6.8 MB/s eta 0:00:02
    --------------------------------------- 0.2/8.9 MB 1.8 MB/s eta 0:00:05
    --------------------------------------- 0.2/8.9 MB 1.8 MB/s eta 0:00:05
    --------------------------------------- 0.2/8.9 MB 1.2 MB/s eta 0:00:08
   - -------------------------------------- 0.3/8.9 MB 1.5 MB/s eta 0:00:06
   - ---------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import json
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
# Încărcăm datele din JSON
try:
    with open('lab4_docs/tesco_sample.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Products in dataset: {len(data)}.")
except FileNotFoundError:
    print("File not found")
    data = []

Products in dataset: 266.


In [25]:
product_names = []
cleaned_descriptions = []

print("Processing...")

for item in data:
    # Extract description
    raw_desc = item.get('description', '')
    name = item.get('name', 'Unknown Product')
    
    # html tags removal
    soup = BeautifulSoup(raw_desc, "html.parser")
    text_desc = soup.get_text(separator=" ")
    
    product_names.append(name)
    cleaned_descriptions.append(text_desc)

print("Text is now clean.")

Processing...
Text is now clean.


In [26]:
# initialize vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# transform descriptions in a matrix
tfidf_matrix = vectorizer.fit_transform(cleaned_descriptions)

print(f"Matrix created. Dimensions: {tfidf_matrix.shape}")

Matrix created. Dimensions: (266, 4215)


In [27]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

print("Similarity matrix calculated.")

Similarity matrix calculated.


In [28]:
max_score = 0
best_pair = (0, 0)

# find maximum in the superior triangle of the matrix
for i in range(len(cosine_sim_matrix)):
    for j in range(i + 1, len(cosine_sim_matrix)):
        score = cosine_sim_matrix[i][j]
        if score > max_score:
            max_score = score
            best_pair = (i, j)

idx1, idx2 = best_pair

print(f"max similarity score: {max_score:.4f}")
print(f"Prod 1: {product_names[idx1]}")
print(f"Prod 2: {product_names[idx2]}")

max similarity score: 1.0000
Prod 1: Unibond Sealant Re-New
Prod 2: Unibond Sealant Re-New


In [ ]:
# writing the similarity matrix in a readme file
with open("similarity_matrix.txt", "w", encoding="utf-8") as f:
    for row in cosine_sim_matrix:
        row_str = "\t".join(f"{value:.4f}" for value in row)
        f.write(row_str + "\n")